# AIMA Post processing

In [ ]:
import cv2

vp = "/content/drive/MyDrive/AIMA/AIMA-DATA/vid_s1_T2.avi"

def check_video(vp):
  
  print("Input:",vp)
  cap = cv2.VideoCapture(vp)
  frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
  print("Number of frames:", frames )
  fps = cap.get(cv2.CAP_PROP_FPS)
  print("FPS:",fps)
  ts = round(frames/fps, 2)
  tm = round((frames/fps) / 60, 2)
  print("Video Duration (s):", ts,'seconds')
  print("Video Duration (m):", tm,'mins')

  return frames, fps, ts

frames, fps, ts = check_video(vp)

In [ ]:
# input - o1
def predict(pulse_pred, fps):

  print("#"*30)
  with open('/content/hr_o1.npy', 'wb') as f:
    np.save(f, np.array(pulse_pred))
  print(pulse_pred)
  print("#"*30)

  pulse_pred = detrend(np.cumsum(pulse_pred), 100)

  print("#"*30)
  with open('/content/hr_o2.npy', 'wb') as f:
    np.save(f, np.array(pulse_pred))
  print(pulse_pred)
  print("#"*30)

  [b_pulse, a_pulse] = butter(1, [0.75 / fps * 2, 2.5 / fps * 2], btype='bandpass')
  pulse_pred = scipy.signal.filtfilt(b_pulse, a_pulse, np.double(pulse_pred))

  print("#"*30)
  with open('/content/hr_o3.npy', 'wb') as f:
    np.save(f, np.array(pulse_pred))
  print(pulse_pred)
  print("#"*30)

predict(pulse_pred, fps)

In [ ]:
import numpy as np
import pandas as pd

def numpy_arrs():
  with open('/content/hr_o1.npy', 'rb') as f:
    o1 = np.load(f)

  with open('/content/hr_o2.npy', 'rb') as f2:
    o2 = np.load(f2)

  with open('/content/hr_o3.npy', 'rb') as f3:
    o3 = np.load(f3)

  o1_plot = plt.plot(o1)
  o3_plot = plt.plot(o3)

  df = pd.read_csv('/content/drive/MyDrive/AIMA/AIMA-DATA/bvp_s1_T2.csv')
  arr_csv = df.to_numpy()

  return o1,o2,o3,arr_csv

o1,o2,o3,arr_csv = numpy_arrs()

In [ ]:
def get_rfft_hr(signal, fps):

  min_freq = 0.9 #
  max_freq = 3 #
  fft_spec = []
  signal_length = len(signal)
  signal = signal.flatten()
  fft_values = np.fft.rfft(signal) # FFT
  fft_magnitudes = np.abs(fft_values)

  frequencies = np.fft.rfftfreq(signal_length, 1./fps) # Frequency data

  indices = np.where((frequencies < min_freq) | (frequencies > max_freq) )[0]
  fft_magnitudes[indices] = 0
  bps_freq=60.0*frequencies
  max_index = np.argmax(fft_magnitudes)
  fft_magnitudes[max_index] = fft_magnitudes[max_index]**2
  fft_spec.append(fft_magnitudes)
  HR =  bps_freq[max_index]

  return HR

heart_rate_vid = get_rfft_hr(o3, fps)
heart_rate_csv = get_rfft_hr(arr_csv, fps)
print("heart_rate_video:", heart_rate_vid)
print("heart_rate_csv:", heart_rate_csv)